In [28]:
     ######clarify variables and files########
%reset -f
!rm *

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [29]:
#importing main librarries
import pandas as pd
from pandas.io.formats.info import DataFrameTableBuilderVerbose
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import scipy as sp
import sklearn
import io
import xgboost

In [30]:
                   ######seprating features and targets#######
# 
# from keras.src.engine.training import potentially_ragged_
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

df_1 = pd.read_csv('Data_for_project_kam.csv')
print(df_1)

#seeking for NaNs
print(df_1.isnull().sum())


  #cleaning NaNs
df_1 = df_1.replace('',np.nan)
df_1 = df_1.dropna(axis="rows", how="any")
df_1 = df_1.replace('',np.nan)
print(df_1.isnull().sum())
df_1.shape
 #declaring fetures and outputs
features = ['condensor_pressure(atm)', 'produced_acetic_acid_mass_fraction', 'reboiler(pot)_temperature(C)']  # 'condenser_heat_removal(Watt)']
targets = ['reflux_ratio', 'daily_work(min)']
# scale = StandardScaler()
X = df_1[features]
Y = df_1[targets]
X_1 = df_1[['condensor_pressure(atm)', 'produced_acetic_acid_mass_fraction']]
X_2 = df_1['reboiler(pot)_temperature(C)']

# x = scale.fit_transform(X)
# y = scale.fit_transform(Y)
X = np.array(X)
Y = np.array(Y)
# print(x)
# print(y)
print(X)
print(Y)
# print(type(x))
print(type(X))

# model_1 = RandomForestRegressor()
# # model_1 = xgb.XGBRegressor()
# # model_1.fit(np.array(X_1).reshape(-1,1),np.array(X_2).reshape(-1,1))
# model_1.fit(X_1,X_2)
# X_2 = model_1.predict(X_1)
# new_X_2 = pd.DataFrame(X_2,columns=["new_reboiler(pot)_temperature(C)"])
# df_1.__delitem__('reboiler(pot)_temperature(C)')
# df_1.insert(2, 'new_reboiler(pot)_temperature(C)', new_X_2)
# print(df_1)
X = df_1[['condensor_pressure(atm)', 'produced_acetic_acid_mass_fraction', 'reboiler(pot)_temperature(C)','condensor_temperature(C)', 'upper_product_flow_rate(kg/hr)', 'condensor_mass_flow_outlet_rate',
          'aceticindutyoverflow', 'order',
          'pot_pressure(atm)', 'fraction_over_pressure', 'duty_over_frac_multiplyed_by_pressure', 'reboiler_temp_on_flowrate', 'condensor_temp_on_flowrate', 'reboiler(pot)_heat_duty(Watt)']]# 'condenser_mass_flow_outlet_rate']]
Y = df_1[['reflux_ratio', 'daily_work(min)']]
def pressure_to_boiler_temp(inps):
  df_1 = pd.read_csv('Data_for_project_kam.csv')
  X_1 = df_1[['condensor_pressure(atm)', 'reboiler(pot)_heat_duty(Watt)', 'produced_acetic_acid_mass_fraction']]
  X_2 = df_1['reboiler(pot)_temperature(C)']
  model = RandomForestRegressor()
  model.fit(X_1,X_2)
  input_data_as_numpy_array = np.asarray(inps)
  input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
  boiler_temp = model.predict(input_data_reshaped)
  return boiler_temp

    ## این جا نیاز نیست یه بار رگرسیون بگیری. با رگرسیون مرتبه 5 که توی اکسله بیا اینجا اول فشار هارو
    ## تبدیل به np.array کن و سپس ببر توی حلقه فوذ ورگرسیون بگیر.
    ## و سپس دوباره اون هارو به دیتا فریم تبدیل کن و بدش به df_1


      condensor_pressure(atm)  reboiler(pot)_heat_duty(Watt)  reflux_ratio  \
0                         0.1                          466.7           0.2   
1                         0.1                          466.7           0.4   
2                         0.1                          466.7           0.6   
3                         0.1                          466.7           0.8   
4                         0.1                          466.7           1.0   
...                       ...                            ...           ...   
1295                      1.0                          933.3          12.2   
1296                      1.0                          933.3          12.4   
1297                      1.0                          933.3          12.6   
1298                      1.0                          933.3          12.8   
1299                      1.0                          933.3          13.0   

      input_mass_fraction_of_acetic_acid  pot_pressure(atm)  da

In [32]:
from keras.models import model_from_json, load_model
import pickle
import joblib

# load model for reboiler duty predictions
filename = "sklearn_heat_duty_nnclassif_model.joblib"

loaded_model_duties = joblib.load(filename)
print("Loaded first sklearn model for reboiler temperature from disk")


##load json and create second dense model to predict and evaluate upper product flow, and reflux ratio
json_file_second_reg = open('flow_and_reflux_dense_model.json', 'r')
loaded_model_json_second_reg = json_file_second_reg.read()
json_file_second_reg.close()
loaded_model_second = model_from_json(loaded_model_json_second_reg)
##load weights into new model
loaded_model_second.load_weights("flow_and_reflux_dense_model.h5")
print("Loaded second deep model for reboiler temperature from disk")

##load json and create second dense model to predict and evaluate reboiler T
json_file_third_reg = open('reb_temperature_dense_model.json', 'r')
loaded_model_json_third_reg = json_file_third_reg.read()
json_file_third_reg.close()
loaded_model_third = model_from_json(loaded_model_json_third_reg)
##load weights into new model
loaded_model_third.load_weights("reb_temperature_dense_model.h5")
print("Loaded second deep model for reboiler temperature from disk")

##load json and create second dense model to predict and evaluate condenser T
json_file_fourth_reg = open('cond_temperature_dense_model.json', 'r')
loaded_model_json_fourth_reg = json_file_fourth_reg.read()
json_file_fourth_reg.close()
loaded_model_fourth = model_from_json(loaded_model_json_fourth_reg)
##load weights into new model
loaded_model_fourth.load_weights("cond_temperature_dense_model.h5")
print("Loaded second deep model for condenser temperature from disk")

##load json and create model
# json_file_third_reg = open('cond_P_reboiler_T_dense_model.json', 'r')
# loaded_model_json_third_reg = json_file_third_reg.read()
# json_file_third_reg.close()
# loaded_model_third = model_from_json(loaded_model_json_third_reg)
# ##load weights into new model
# loaded_model_third.load_weights("cond_P_reboiler_T_dense_model.h5")
# print("Loaded third deep model for reflux ratio from disk")

# loading scalers
file_name_1 = "scaler_for_first_input.pkl"
file_name_2 = "scaler_for_second_input.pkl"
# file_name_2_reb = "scaler_for_second_input_reb.pkl"
file_name_2_output = "scaler_for_second_output.pkl"
file_name_3_input = "scaler_for_third_input.pkl"
file_name_3_output = "scaler_for_third_output_reb.pkl"
file_name_4_output = "scaler_for_fourth_output_cond.pkl"




# load
first_loaded_scaler = pickle.load(open(file_name_1, "rb"))
second_loaded_scaler = pickle.load(open(file_name_2, "rb"))
# second_loaded_scaler_reb = pickle.load(open(file_name_2_reb, "rb"))
second_loaded_scaler_output = pickle.load(open(file_name_2_output, "rb"))
third_loaded_scaler_input = pickle.load(open(file_name_3_input, "rb"))
third_loaded_scaler_output = pickle.load(open(file_name_3_output, "rb"))
fourth_loaded_scaler_output = pickle.load(open(file_name_4_output, "rb"))
# second_loaded_scaler_output = pickle.load(open(file_name_3, "rb"))


print("scalers are in")



# # ##load json and create model
# # json_file_fourth_reg = open('forth_regression_model_LSTM.json', 'r')
# # loaded_model_json_fourth_reg = json_file_fourth_reg.read()
# # json_file_fourth_reg.close()
# # loaded_model_fourth = model_from_json(loaded_model_json_fourth_reg)
# # ##load weights into new model
# # loaded_model_fourth.load_weights("forth_regression_model_LSTM.h5")
# # print("Loaded fourth deep model for condensor temperature from disk")

# json_file_fourth_reg = open('cond_temp_dense_model.json', 'r')
# loaded_model_json_fourth_reg = json_file_fourth_reg.read()
# json_file_fourth_reg.close()
# loaded_model_fourth = model_from_json(loaded_model_json_fourth_reg)
# ##load weights into new model
# loaded_model_fourth.load_weights("cond_temp_dense_model.h5")
# print("Loaded fourth deep model for condensor temperature from disk")

# json_file_fifth_reg = open('order_model.json', 'r')
# loaded_model_json_fifth_reg = json_file_fifth_reg.read()
# json_file_fifth_reg.close()
# loaded_model_fifth = model_from_json(loaded_model_json_fifth_reg)
# ##load weights into new model
# loaded_model_fifth.load_weights("order_model.h5")
# print("Loaded fifth deep model for acetic acid order from disk")

Loaded first sklearn model for reboiler temperature from disk
Loaded second deep model for reboiler temperature from disk
Loaded second deep model for reboiler temperature from disk
Loaded second deep model for condenser temperature from disk
scalers are in


In [21]:
# for i, layer in enumerate (loaded_model_first.layers):
#     print (i, layer)
#     try:
#         print ("    ",layer.activation)
#     except AttributeError:
#         print('   no activation attribute')
# loaded_model_first.summary()

0 <keras.layers.core.dense.Dense object at 0x0000012090C10700>
     <function linear at 0x000001208502A8C0>
1 <keras.layers.core.dense.Dense object at 0x0000012090C10850>
     <function relu at 0x0000012085029EA0>
2 <keras.layers.core.dense.Dense object at 0x0000012090C109A0>
     <function relu at 0x0000012085029EA0>
3 <keras.layers.core.dense.Dense object at 0x0000012090C24880>
     <function relu at 0x0000012085029EA0>
4 <keras.layers.core.dense.Dense object at 0x0000012090C73E80>
     <function linear at 0x000001208502A8C0>
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 18)                108       
                                                                 
 dense_36 (Dense)            (None, 18)                342       
                                                                 
 dense_37 (Dense)            (None, 18)         

In [17]:
for i, layer in enumerate (loaded_model_second.layers):
    print (i, layer)
    try:
        print ("    ",layer.activation)
    except AttributeError:
        print('   no activation attribute')
loaded_model_second.summary()

0 <keras.layers.core.dense.Dense object at 0x0000016BCC5BDE10>
     <function linear at 0x0000016BC2ADA950>
1 <keras.layers.core.dense.Dense object at 0x0000016BCC5BD030>
     <function relu at 0x0000016BC2AD9F30>
2 <keras.layers.core.dense.Dense object at 0x0000016BCC5BD450>
     <function relu at 0x0000016BC2AD9F30>
3 <keras.layers.core.dense.Dense object at 0x0000016BCC5A4D90>
     <function relu at 0x0000016BC2AD9F30>
4 <keras.layers.core.dense.Dense object at 0x0000016BC5F58C70>
     <function relu at 0x0000016BC2AD9F30>
5 <keras.layers.core.dense.Dense object at 0x0000016BCC5A0A00>
     <function linear at 0x0000016BC2ADA950>
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 28)                252       
                                                                 
 dense_86 (Dense)            (None, 28)                812       
       

In [33]:
# 'daily_work(min)', 'produced_acetic_acid_mass_fraction', 'order', 'condensor_pressure(atm)'
# 'reboiler(pot)_heat_duty(Watt)'


fraction = float(input("please insert the fraction of acetic acid that you desire in distillate (between 0.992000 and 1.00000): "))
condenser_pressure = float(input("please insert the condenser pressure in atmosphere: "))
daily = float(input("please insert daily work period in minutes(between 54 and 633)"))
pot_pressure = condenser_pressure + 0.098692326671601
order_of_purification = -math.log10(1 - fraction)
# daily_work_multip_duty = daily * reboiler_duty
# duty_over_frac_multiplyed_by_pressure = reboiler_duty / (fraction * condenser_pressure)
# fraction_over_pressure = fraction / condenser_pressure
# pot_pressure = condenser_pressure + 0.098692326671601

# elif heat_duty_select == 1:
#   reboiler_duty = 233.3
#   daily = float(input("please insert daily work period in minutes(between 129 and 1210)"))
# elif heat_duty_select == 3:
#   reboiler_duty = 700
#   daily = float(input("please insert daily work period in minutes(between 63 and 424)"))
 
 
inputs_testing_for_flows = (daily, fraction, order_of_purification, condenser_pressure)
inputs_testing_for_flows = np.asarray(inputs_testing_for_flows)
inputs_testing_for_flows = inputs_testing_for_flows.reshape(1,-1)
inputs_testing_for_flows = first_loaded_scaler.transform(inputs_testing_for_flows)

suggested_reboiler_duty_labeled = loaded_model_duties.predict(inputs_testing_for_flows)

if suggested_reboiler_duty_labeled[0] == 0:
  suggested_reboiler_duty = 466.7
elif suggested_reboiler_duty_labeled[0] == 1:
  suggested_reboiler_duty = 933.3
  
# daily_work(min)', 'produced_acetic_acid_mass_fraction', 'order', 'condensor_pressure(atm)', 'reboiler(pot)_heat_duty(Watt)',
# 'daily_work_multip_duty','fraction_over_pressure',	'duty_over_frac_multiplyed_by_pressure'
#
# 'condensor_temperature(C)','reboiler(pot)_temperature(C)','reflux_ratio'

# daily_work_multip_duty = daily * suggested_reboiler_duty
# fraction_over_pressure = fraction / condenser_pressure
# duty_over_fraction_multip_pressure = suggested_reboiler_duty / (fraction * condenser_pressure)
  
inputs_testing_for_flow_and_reflux = (daily, fraction, order_of_purification, condenser_pressure, suggested_reboiler_duty)#, daily_work_multip_duty, fraction_over_pressure, duty_over_fraction_multip_pressure)
inputs_testing_for_flow_and_reflux = np.asarray(inputs_testing_for_flow_and_reflux )
inputs_testing_for_flow_and_reflux = inputs_testing_for_flow_and_reflux .reshape(1,-1)
inputs_testing_for_flow_and_reflux = second_loaded_scaler.transform(inputs_testing_for_flow_and_reflux )

suggested_flow_and_reflux = second_loaded_scaler_output.inverse_transform(loaded_model_second.predict(inputs_testing_for_flow_and_reflux ))
suggested_flow = suggested_flow_and_reflux[0][0]
suggested_reflux = suggested_flow_and_reflux[0][1]

inputs_testing_for_reb_T = (daily, fraction, order_of_purification, condenser_pressure, suggested_reboiler_duty)
inputs_testing_for_reb_T = np.asarray(inputs_testing_for_reb_T)
inputs_testing_for_reb_T = inputs_testing_for_reb_T.reshape(1,-1)
inputs_testing_for_reb_T = third_loaded_scaler_input.transform(inputs_testing_for_reb_T)

suggested_reb_T = third_loaded_scaler_output.inverse_transform(loaded_model_third.predict(inputs_testing_for_reb_T))
suggested_reboiler_T = suggested_reb_T[0][0]

inputs_testing_for_cond_T = (daily, fraction, order_of_purification, condenser_pressure, suggested_reboiler_duty)
inputs_testing_for_cond_T = np.asarray(inputs_testing_for_cond_T)
inputs_testing_for_cond_T = inputs_testing_for_cond_T.reshape(1,-1)
inputs_testing_for_cond_T = third_loaded_scaler_input.transform(inputs_testing_for_cond_T)

suggested_condenser_T = fourth_loaded_scaler_output.inverse_transform(loaded_model_fourth.predict(inputs_testing_for_cond_T))
suggested_condenser_T = suggested_condenser_T[0][0]

if suggested_flow <= 0:
  print("reflux ratio will be to high")
else:
  print("acetic acid fraction in product is:", fraction)
  print("condenser pressure is", condenser_pressure,"atmosphere")
  print("pot pressure is", pot_pressure,"atmosphere")
  print("daily work period is", daily,"minutes")
  print("reboiler duty is", suggested_reboiler_duty,"watt")
  print("reboiler temperature is", suggested_reboiler_T, "degrees Celsius")
  print("condenser temperature is", suggested_condenser_T, "degrees Celsius") 
  print("reflux ratio is", suggested_reflux) 
  print("upper product flow rate is", suggested_flow) 

# # print("acetic acid fraction in product after all", acetic_acid_fraction.reshape(-1).transpose()[0]) 

# print("upper product flow rate is", suggested_upper_flow_rate.reshape(-1).transpose()[0], "kg/hr")




  




1/1 [==============================] - ETA: 0s

c:\Users\Lenovo\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Lenovo\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Lenovo\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Lenovo\python\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 41ms/step
reflux ratio will be to high


In [34]:
print("acetic acid fraction in product is:", fraction)
print("condenser pressure is", condenser_pressure,"atmosphere")
print("pot pressure is", pot_pressure,"atmosphere")
print("daily work period is", daily,"minutes")
print("reboiler duty is", suggested_reboiler_duty,"watt")
print("reboiler temperature is", suggested_reboiler_T, "degrees Celsius")
print("condenser temperature is", suggested_condenser_T, "degrees Celsius") 
print("reflux ratio is", suggested_reflux) 
print("upper product flow rate is", suggested_flow) 

acetic acid fraction in product is: 0.999
condenser pressure is 14.0 atmosphere
pot pressure is 14.098692326671602 atmosphere
daily work period is 400.0 minutes
reboiler duty is 933.3 watt
reboiler temperature is 227.45259 degrees Celsius
condenser temperature is 240.19579 degrees Celsius
reflux ratio is 21.564137
upper product flow rate is -0.24237682


In [14]:
print(suggested_reboiler_duty)


933.3


In [15]:
print(suggested_Ts)

[[-0.08146319 -0.22839257]]


In [14]:
print(    inputs_testing_for_flow_and_reflux )

[[ 0.85867909  0.3062315  -0.48228555 29.41912386  1.          2.69976843
  -1.07113011 -0.94615827]]


In [91]:
1 -  (10** - int(order_of_purification))

0.99

In [164]:
# regressor_main.fit(X,Y)

## 2	2.098692327	466.7	0.99	3.972966667	165.749064	163.0687495	0.499823	233.4326352	99.9646	535.1616894		0.999646	144.4940959	0.871764174	3.030305306	142.1574937	1.000000751	4.358820871
## 4  238.378

# [['condensor_pressure(atm)', 'produced_acetic_acid_mass_fraction', 'reboiler(pot)_temperature(C)', 'reflux_ratio', 'upper_product_flow_rate(kg/hr)', 'condensor_mass_flow_outlet_rate',
#           'aceticindutyoverflow',
#           'pot_pressure(atm)', 'fraction_over_pressure', 'duty_over_frac_multiplyed_by_pressure', 'reboiler_temp_on_flowrate', 'condensor_temp_on_flowrate', 'reboiler(pot)_heat_duty(Watt)']]

inputs = (condensor_pressure, fraction, reboiler_temp, reflux_ratio, upper_product_flow_rate, upper_product_flow_rate*(1+reflux_ratio) ,fraction*reboiler_duty/upper_product_flow_rate , pot_pressure, fraction/condensor_pressure, duty_over_fraction_multip_pressure, reboiler_temp/upper_product_flow_rate, condensor_temp/upper_product_flow_rate, reboiler_duty)
input_data_as_numpy_array = np.asarray(inputs, dtype=float)
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
loaded_model.predict(input_data_reshaped)
# print(results)
#'atmosphere_pressure', 'reboiler(pot)_heat_duty(Watt)', 'produced_acetic_acid_mass_fraction', 'new_reboiler(pot)_temperature(C)', 'upper_product_flow_rate(kg/hr)', 'pot_pressure(atm)', 'condenser_mass_flow_outlet_rate

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1008\2017386810.py:10: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  input_data_as_numpy_array = np.asarray(inputs, dtype=float)


1/1 [==============================] - 0s 134ms/step


array([[ 13.159039, 621.89606 ]], dtype=float32)

In [55]:
X = df_1[['condensor_pressure(atm)', 'reboiler(pot)_temperature(C)','condensor_temperature(C)', 'upper_product_flow_rate(kg/hr)', 'condensor_mass_flow_outlet_rate',
          'pot_pressure(atm)', 'reboiler(pot)_heat_duty(Watt)']]
Y = df_1[['order']]

In [56]:
from sklearn.model_selection import train_test_split

X_dense = np.array(X)
Y_dense = np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X_dense, Y_dense , test_size=0.2, random_state=6)

In [79]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split

X_dense = np.array(X)
Y_dense = np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X_dense, Y_dense , test_size=0.2, random_state=6)
n_inputs_sequ = X_train.shape[1]
n_outputs_sequ = Y_train.shape[1]

mainman_model = Sequential()
# mainman_model.add(LSTM(25, recurrent_dropout=0.2, activation='hard_sigmoid', input_shape=(n_inputs_sequ,1), return_sequences=True))
# mainman_model.add(LSTM(30, recurrent_dropout=0.2, activation='relu', return_sequences=True))
# mainman_model.add(LSTM(24, recurrent_dropout=0.2, activation='relu', return_sequences=True))
# mainman_model.add(LSTM(24, recurrent_dropout=0.2, activation='relu', return_sequences=True))
# mainman_model.add(LSTM(24, recurrent_dropout=0.2, activation='relu'))

mainman_model.add(Dense(60, kernel_initializer='uniform', activation='relu', input_dim=n_inputs_sequ))
mainman_model.add(Dense(48, kernel_initializer='uniform', activation='relu'))
mainman_model.add(Dense(36, kernel_initializer='uniform', activation='relu'))
mainman_model.add(Dense(18, kernel_initializer='uniform', activation='relu'))
mainman_model.add(Dense(n_outputs_sequ, activation='linear'))
opt = optimizers.Adam(learning_rate=0.01, beta_1=0.8, beta_2=0.75, epsilon=1e-7, amsgrad=False, name='Adam')

mainman_model.compile(loss='mae', optimizer='adam')
mainman_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 60)                480       
                                                                 
 dense_27 (Dense)            (None, 48)                2928      
                                                                 
 dense_28 (Dense)            (None, 36)                1764      
                                                                 
 dense_29 (Dense)            (None, 18)                666       
                                                                 
 dense_30 (Dense)            (None, 1)                 19        
                                                                 
Total params: 5,857
Trainable params: 5,857
Non-trainable params: 0
_________________________________________________________________


In [16]:
# from sklearn.ensemble import RandomForestRegressor
# rfreg = RandomForestRegressor()
# rfreg.fit(X_train, Y_train)


In [44]:
# rf_acet = rfreg.predict(X_test)

In [85]:
# Fitting the data
history = mainman_model.fit(X_dense,
                    Y_dense,
                    #shuffle = False, # Since this is time series data
                    epochs=500,
                    batch_size=200,
                    validation_split=0.2,
                    verbose=1) # Verbose outputs data

Epoch 1/500
6/6 [==============================] - 0s 13ms/step - loss: 0.0923 - val_loss: 0.3261
Epoch 2/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0798 - val_loss: 0.3588
Epoch 3/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0807 - val_loss: 0.3680
Epoch 4/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0817 - val_loss: 0.3178
Epoch 5/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0799 - val_loss: 0.3285
Epoch 6/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0785 - val_loss: 0.3382
Epoch 7/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0781 - val_loss: 0.3341
Epoch 8/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0758 - val_loss: 0.3646
Epoch 9/500
6/6 [==============================] - 0s 8ms/step - loss: 0.0799 - val_loss: 0.3967
Epoch 10/500
6/6 [==============================] - 0s 9ms/step - loss: 0.0755 - val_loss: 0.4169
Epoch 11/500
6/6 [==========

In [86]:
X_test_newa = pd.DataFrame(X_test)
Y_pred_general = mainman_model.predict(X_test_newa.tail(60))

Y_predicted_cond = pd.DataFrame(Y_pred_general.transpose()[0])
# Y_predicted_daily_work = pd.DataFrame(Y_pred_general.transpose()[1])
Y_test_new = pd.DataFrame(Y_test).tail(60)
Y_test_cond = pd.DataFrame(Y_test_new.values.transpose()[0])
# Y_test_daily_work = pd.DataFrame(Y_test_new.values.transpose()[1])
pd.DataFrame({'real reflux ratio': Y_test_cond.squeeze(),
            #   'predicted reflux ratio': Y_predicted_reflux_ratio.squeeze(),
            #   'real daily work time': Y_test_daily_work.squeeze(),
              'predicted daily work time' : Y_predicted_cond.squeeze().values})

2/2 [==============================] - 0s 4ms/step


,real reflux ratio,predicted daily work time
0,2.2,2.371576
1,3.9,4.313326
2,2.2,2.444408
3,2.5,2.493740
4,2.4,2.430295
5,4.3,4.410135
6,6.0,5.772016
7,6.0,6.016432
8,3.9,4.053036
9,4.7,4.909290


In [87]:
##serialize model to YAML
# from keras.models import model_from_json, load_model

# model_json = mainman_model.to_json()
# with open("order_model.json", "w") as json_file:
#     json_file.write(model_json)
    
# ##serialize weights to HDF5
# mainman_model.save_weights("order_model.h5")
# print("Saved model to disk")

Saved model to disk


In [29]:
import xgboost as xgb

regressor_xgb = xgb.XGBRegressor(n_estimators=5001)#, subsample=0.99, learning_rate=0.0061, min_child_weight=3, seed=32, max_depth=9, nthread=2)#n_estimators=30001,
                                #   early_stop_rounds=50,#,
                                #   max_depth=4)
#                                   min_child_weight=7,
#                                   gamma=0,
#                                   subsample=0.9,
#                                   colsample_bytree=1,
#                                   reg_alpha=0.05,
#                                   nthread=5,
#                                   scale_pos_weight=1,
#                                   objective='reg:squarederror',
#                                   seed=32,
#                                   learning_rate=0.00299)

# # regressor_main = RandomForestRegressor()
regressor_xgb.fit(X_train, Y_train, eval_set=[(X_train, Y_train),(X_test, Y_test)],verbose=2000)

[0]	validation_0-rmse:0.10144	validation_1-rmse:0.10278
[2000]	validation_0-rmse:0.00111	validation_1-rmse:0.01722
[4000]	validation_0-rmse:0.00111	validation_1-rmse:0.01722
[5000]	validation_0-rmse:0.00111	validation_1-rmse:0.01722


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5001, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [30]:
X_test_newa = pd.DataFrame(X_test)
Y_pred_general = regressor_xgb.predict(X_test_newa.tail(60))

Y_predicted_cond = (Y_pred_general.transpose()[0])
# Y_predicted_daily_work = pd.DataFrame(Y_pred_general.transpose()[1])
Y_test_new = pd.DataFrame(Y_test).tail(60)
Y_test_cond = pd.DataFrame(Y_test_new.values.transpose()[0])
# Y_test_daily_work = pd.DataFrame(Y_test_new.values.transpose()[1])
pd.DataFrame({'real reflux ratio': Y_test_cond.squeeze(),
            #   'predicted reflux ratio': Y_predicted_reflux_ratio.squeeze(),
            #   'real daily work time': Y_test_daily_work.squeeze(),
              'predicted daily work time' : Y_predicted_cond.squeeze()})

c:\Users\Lenovo\python\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Lenovo\python\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Lenovo\python\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Lenovo\python\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


,real reflux ratio,predicted daily work time
0,99.2998,99.310211
1,99.9864,99.310211
2,99.3529,99.310211
3,99.6532,99.310211
4,99.5937,99.310211
5,99.9955,99.310211
6,99.9999,99.310211
7,99.9999,99.310211
8,99.9883,99.310211
9,99.9979,99.310211


In [17]:
# condensor_pressure = suggested_pressure

In [18]:
# pot_pressure = condensor_pressure + 0.098692326671601
# daily_work_multip_duty = reboiler_duty * daily
# fraction_over_condensor_pressure = fraction / condensor_pressure
# daily_work_multip_fraction_over_duty = daily * fraction / reboiler_duty
# duty_over_fraction_multip_pressure = reboiler_duty / (fraction * condensor_pressure)

In [29]:
# print("acetic acid fraction in product is:", fraction)
# print("reboiler duty is", reboiler_duty,"watt")
# print("daily column work period is", daily,"minutes")
# print("condensor pressure is", condensor_pressure.transpose()[0],"atmosphere")
# print("pot pressure is", pot_pressure.transpose()[0],"atmosphere")

In [28]:
# from keras.models import model_from_json, load_model

# ##load json and create model
# json_file_first_reg = open('first_regression_model_LSTM.json', 'r')
# loaded_model_json_first_reg = json_file_first_reg.read()
# json_file_first_reg.close()
# loaded_model_first = model_from_json(loaded_model_json_first_reg)
# ##load weights into new model
# loaded_model_first.load_weights("first_regression_model_LSTM.h5")
# print("Loaded model from disk")


In [113]:
# print(type(float(suggested_daily)))
# print(type(fraction))
# print(float(suggested_daily))

In [27]:
# ###### input for user
# inputs_testing = (fraction, reboiler_duty, daily, daily_work_multip_duty, daily_work_multip_fraction_over_duty, condensor_pressure)
# input_data_as_numpy_array_first_reg = np.asarray(inputs_testing, dtype=float)
# input_data_reshaped_testing = input_data_as_numpy_array_first_reg.reshape(1, -1)
# upper_product_flow_rate = loaded_model_first.predict(input_data_reshaped_testing)
# print("upper product flow rate is", upper_product_flow_rate.reshape(-1).transpose()[0], "kg/hr")

In [30]:
# from keras.models import model_from_json, load_model

# ##load json and create model
# json_file_second_reg = open('second_regression_model_LSTM_only.json', 'r')
# loaded_model_json_second_reg = json_file_second_reg.read()
# json_file_second_reg.close()
# loaded_model_second = model_from_json(loaded_model_json_second_reg)
# ##load weights into new model
# loaded_model_second.load_weights("second_regression_model_LSTM_only.h5")
# print("Loaded model from disk")

In [31]:
# from keras.models import model_from_json, load_model

# ##load json and create model
# json_file_third_reg = open('third_regression_model.json', 'r')
# loaded_model_json_second_reg = json_file_third_reg.read()
# json_file_third_reg.close()
# loaded_model_third = model_from_json(loaded_model_json_second_reg)
# ##load weights into new model
# loaded_model_third.load_weights("third_regression_model.h5")
# print("Loaded model from disk")

In [32]:
# #[['produced_acetic_acid_mass_fraction', 'reboiler(pot)_heat_duty(Watt)', 'condensor_pressure(atm)', 'pot_pressure(atm)','fraction_over_pressure', 'duty_over_frac_multiplyed_by_pressure']]
# ###### input for user
# inputs_testing_first = (fraction, reboiler_duty,condensor_pressure, pot_pressure, fraction_over_condensor_pressure, duty_over_fraction_multip_pressure)
# input_data_as_numpy_array_first_reg = np.asarray(inputs_testing_first, dtype=float)
# input_data_reshaped_testing = input_data_as_numpy_array_first_reg.reshape(1, -1)
# reboiler_temp = loaded_model_second.predict(input_data_reshaped_testing)
# print("reboiler temperature is", reboiler_temp.reshape(-1).transpose()[0], "degrees Celsius")

In [139]:
# condensor_pressure
# upper_product_flow_rate.reshape(-1).transpose()[0]
# # reboiler_temp.reshape(-1).transpose()[0]

In [33]:
# #[['produced_acetic_acid_mass_fraction', 'reboiler(pot)_heat_duty(Watt)', 'daily_work(min)', 'daily_work_multip_duty', 'daily_work_multip_fraction_over_duty', 'condensor_pressure(atm)','predicted_upper_product_flow_rate(kg/hr)', 'predicted_reboiler(pot)_temperature(C)']]
# ###### input for user
# inputs_testing_second = (fraction, reboiler_duty, daily, daily_work_multip_duty, daily_work_multip_fraction_over_duty, condensor_pressure, upper_product_flow_rate.reshape(-1).transpose()[0], reboiler_temp.reshape(-1).transpose()[0])
# input_data_as_numpy_array_first_reg = np.asarray(inputs_testing_second, dtype=float)
# input_data_reshaped_testing = input_data_as_numpy_array_first_reg.reshape(1, -1)
# reflux_ratio = loaded_model_third.predict(input_data_reshaped_testing)
# print("reflux ratio is", reflux_ratio.reshape(-1).transpose()[0])

In [34]:
# print("acetic acid fraction in product is:", fraction)
# print("reboiler duty is", reboiler_duty,"watt")
# print("daily column work period is", daily,"minutes")
# print("condensor pressure is", condensor_pressure.transpose()[0],"atmosphere")
# print("pot pressure is", pot_pressure.transpose()[0],"atmosphere")
# print("upper product flow rate is", upper_product_flow_rate.reshape(-1).transpose()[0], "kg/hr")
# print("reboiler temperature is", reboiler_temp.reshape(-1).transpose()[0], "degrees Celsius")
# print("reflux ratio is", reflux_ratio.reshape(-1).transpose()[0])